In [3]:
# DAY 1 – Advanced Python Foundations
# Module 1: Advanced Python Foundations
# • Deep Dive into Python Internals
# • Memory model and reference counting
# • Garbage collection and object lifecycle
# • Mutable vs immutable objects
# • Understanding GIL (Global Interpreter Lock)

In [ ]:
# Deep Dive into Python Internals

# When we write Python code, we usually focus on syntax and logic. However, beneath the syntax, Python has an execution engine that decides how code is interpreted, how objects are created, how memory is managed, and when statements are executed.

x = 10

# Python parses the code to check syntax correctness.
# The parsed code is compiled into bytecode (a lower-level, platform-independent instruction set).
# The Python Virtual Machine executes this bytecode.

# Everything in Python Is an Object

# Every object in Python has three core properties:
# Type: what kind of object it is
# Identity: a unique identifier during its lifetime
# Value: the data the object represents

# Inspecting Objects at Runtime
x = 42

print(type(x))
print(id(x))
print(x)

# Names and Object References
a = 10
b = a

print(id(a))
print(id(b))

In [5]:
a = 10
b = a

print(id(a))
print(id(b))

a = a+2
print(id(a))
print(id(b))

4342431648
4342431648
4342431712
4342431648


In [ ]:
# Execution Order in Python
# Python executes code in a well-defined order.

# Top-level statements run immediately when the file is executed.
# Function bodies do not run until the function is explicitly called.
# Definitions (functions, classes) are processed at runtime, not ahead of time.


# Why Python Internals Matter
# Why modifying an object inside a function can affect the caller
# Why copying variables does not always copy data
# Why some bugs appear only at runtime
# Why performance optimizations sometimes fail

In [6]:
# Exercise: Execution Awareness
# Create a Python script that demonstrates execution order.

# Requirements:

# Print a message at the top level.
# Define two functions with print statements inside them.
# Call only one of the functions.
# Print a final message at the end.
# Objective:

# Observe which statements execute immediately.
# Observe which statements execute only when called.
# The output should clearly show the difference between definition time and execution time.

In [7]:
# Memory model and reference counting 

# At a high level:

# Memory is allocated for objects, not for variables
# Variables are names bound to objects
# Objects remain in memory as long as they are needed
# Python decides when an object can be safely removed

x = 100

# What is Reference Counting?
# Python uses reference counting to keep track of how many references point to an object.
# When an object's reference count drops to zero, Python deallocates the memory for that object.

# Every object in Python maintains a counter called the reference count.

# The reference count represents:

# How many names refer to the object
# How many data structures contain the object
# How many internal references Python itself holds

In [8]:
import sys # sys module provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter.

a = []
print(sys.getrefcount(a))

2


In [ ]:
# Reference counts increase when:

# A new variable points to the object
# The object is added to a list, dictionary, or other container
# The object is passed as a function argument


# Reference counts decrease when:

# A variable is reassigned
# A variable goes out of scope
# An object is removed from a container
# A function finishes execution

import sys

data = []
print(sys.getrefcount(data))

alias = data
print(sys.getrefcount(data))

del alias
print(sys.getrefcount(data))

# What happens step by step:

# data creates one reference
# alias adds another reference
# Deleting alias removes one reference
# The object remains alive as long as data still exists

In [ ]:
# Scope and Reference Lifetime

import sys

def create_object():
    obj = {}
    print(sys.getrefcount(obj))

create_object()

In [9]:
# Immediate Cleanup Through Reference Counting

# One important characteristic of reference counting is deterministic cleanup.

# When the reference count drops to zero:

# The object is destroyed immediately
# Its memory is released
# Any cleanup logic (such as file handles) is executed right away
# This is why patterns like context managers work reliably in Python.



# Practical Implications in Real Applications
# Understanding reference counting explains many real-world behaviors:

# Why objects passed to functions may stay alive longer than expected
# Why holding references in global variables can cause memory growth
# Why removing objects from lists and dictionaries matters
# Why circular references require special handling

In [ ]:
import sys

class Demo:
    def __init__(self, name):
        self.name = name
        print(f"Object {self.name} created")

    def __del__(self):
        print(f"Object {self.name} destroyed")

obj = Demo("A")
print(sys.getrefcount(obj))

alias = obj
print(sys.getrefcount(obj))

del obj
print("Deleted obj reference")

del alias
print("Deleted alias reference")

print(sys.getrefcount(obj))
# Note: The last line will raise an error since 'obj' has been deleted.

In [10]:
# Memory Model and Reference Counting
# Python’s Memory Model: The Big Picture
# Python manages memory automatically. As a developer, memory allocation and deallocation are not handled manually, but they still follow strict internal rules. Understanding these rules explains why objects stay alive, when they are destroyed, and why some objects disappear immediately while others do not.

# At a high level:

# Memory is allocated for objects, not for variables
# Variables are names bound to objects
# Objects remain in memory as long as they are needed
# Python decides when an object can be safely removed
# The core mechanism that enables this behavior is reference counting.

# Objects Live Independently of Variables
# When an object is created, Python allocates memory for it and keeps track of how many references point to it.

# x = 100
# What happens internally:

# An integer object representing 100 is created (or reused)
# The name x points to that object
# The object now has at least one reference
# The lifetime of the object does not depend on the name x. It depends on how many references exist to that object.

# Reference Count: What It Means
# Every object in Python maintains a counter called the reference count.

# The reference count represents:

# How many names refer to the object
# How many data structures contain the object
# How many internal references Python itself holds
# As long as the reference count is greater than zero, the object must remain in memory.

# When the reference count becomes zero, Python can immediately reclaim the memory.

# Observing Reference Counts
# Python provides a way to inspect reference counts using the sys module.

# import sys

# a = []
# print(sys.getrefcount(a))
# Important note:

# getrefcount() temporarily increases the count by one because the object is passed as an argument
# The actual reference count is usually one less than the printed value
# This tool is meant for understanding behavior, not for production logic.

# How Reference Counts Change
# Reference counts increase when:

# A new variable points to the object
# The object is added to a list, dictionary, or other container
# The object is passed as a function argument
# Reference counts decrease when:

# A variable is reassigned
# A variable goes out of scope
# An object is removed from a container
# A function finishes execution
# Example: Reference Count in Action
# import sys

# data = []
# print(sys.getrefcount(data))

# alias = data
# print(sys.getrefcount(data))

# del alias
# print(sys.getrefcount(data))
# What happens step by step:

# data creates one reference
# alias adds another reference
# Deleting alias removes one reference
# The object remains alive as long as data still exists
# Scope and Reference Lifetime
# References are also affected by scope.

# import sys

# def create_object():
#     obj = {}
#     print(sys.getrefcount(obj))

# create_object()
# Once the function finishes execution:

# The local variable obj goes out of scope
# Its reference is removed
# If no other references exist, the object becomes eligible for destruction
# This explains why temporary objects often disappear immediately after function execution.

# Immediate Cleanup Through Reference Counting
# One important characteristic of reference counting is deterministic cleanup.

# When the reference count drops to zero:

# The object is destroyed immediately
# Its memory is released
# Any cleanup logic (such as file handles) is executed right away
# This is why patterns like context managers work reliably in Python.

# Practical Implications in Real Applications
# Understanding reference counting explains many real-world behaviors:

# Why objects passed to functions may stay alive longer than expected
# Why holding references in global variables can cause memory growth
# Why removing objects from lists and dictionaries matters
# Why circular references require special handling
# Reference counting is efficient, predictable, and fast, which is why Python uses it as the primary memory management strategy.

# Reference Counting Limitation: Circular References
# Reference counting alone cannot handle circular references.

# Example conceptually:

# Object A references Object B
# Object B references Object A
# No external references exist
# Even though the objects are no longer useful, their reference counts never reach zero. Python solves this problem using a separate garbage collection mechanism, which will be covered later.

# At this stage, it is enough to be aware that circular references exist and are handled differently.

# Script-Based Demonstration of Reference Lifetime
# This code must be saved as reference_lifetime_demo.py and executed from the terminal using:

# python reference_lifetime_demo.py
# It should not be run inside a Jupyter Notebook.

# import sys

# class Demo:
#     def __init__(self, name):
#         self.name = name
#         print(f"Object {self.name} created")

#     def __del__(self):
#         print(f"Object {self.name} destroyed")

# obj = Demo("A")
# print(sys.getrefcount(obj))

# alias = obj
# print(sys.getrefcount(obj))

# del obj
# print("Deleted obj reference")

# del alias
# print("Deleted alias reference")
# Observation:

# The destructor runs only when the final reference is removed
# Object destruction timing is deterministic in this case
# Exercise: Tracking Reference Behavior
# Create a Python script that demonstrates reference count changes.

# Requirements:

# Create a custom class with a constructor and destructor
# Create multiple references to the same object
# Delete references one by one
# Print messages showing when the object is destroyed
# Objective:

# Observe how reference count controls object lifetime
# Understand why deletion order matters
# The output should clearly show that the object is destroyed only after the final reference is removed.

In [12]:
 # Garbage collection and object lifecycle
# While reference counting handles most memory management, it cannot deal with circular references. To address this, Python includes a garbage collector that periodically looks for groups of objects that reference each other but are no longer reachable from the program.

# At a high level, the lifecycle consists of:

# Object creation
# Active usage
# Becoming unreachable
# Cleanup and destruction

# Reference counting handles most of this lifecycle automatically. Garbage collection exists to handle the cases where reference counting alone is insufficient.


# When Reference Counting Is Not Enough
# Reference counting works well when objects are released linearly. However, it fails in one specific scenario: circular references.

# A circular reference occurs when:

# Object A references Object B
# Object B references Object A
# No external references exist
# Even though these objects are no longer useful, their reference counts never reach zero. Without additional logic, they would remain in memory forever.

# This is the reason Python includes a garbage collector in addition to reference counting.



# Garbage collection in Python:

# Detects groups of objects that reference each other
# Determines whether those objects are still reachable
# Frees memory for unreachable object cycles
    

# Garbage collection is based on reachability.
# An object is considered reachable if:
# It can be accessed directly or indirectly from a root reference

# Generational Garbage Collection Model
# Python organizes objects into generations to make garbage collection efficient.

# There are three generations:

# Generation 0: Newly created objects
# Generation 1: Objects that survive one garbage collection cycle
# Generation 2: Long-lived objects



# Garbage Collector Triggers
# Garbage collection does not run continuously.

# It is triggered when:

# The number of object allocations crosses certain thresholds
# Python decides it is safe and useful to run a collection cycle

# Observing Garbage Collection Behavior
# Python provides the gc module to inspect and control garbage collection.

import gc # gc module provides an interface to the garbage collection facility.

print(gc.get_threshold()) # get_threshold() returns the current collection thresholds as a tuple of three integers.
print(gc.get_count()) # get_count() returns the current collection counts as a tuple of three integers.

(700, 10, 10)
(261, 3, 8)


In [ ]:
# Circular Reference Example

class Node:
    def __init__(self):
        self.other = None

a = Node()
b = Node()

a.other = b
b.other = a

In [ ]:
# Object Destruction and the __del__ Method¶
# When an object is destroyed, Python may call a special method named __del__.

# Important points:

# __del__ is not guaranteed to run immediately for cyclic objects
# Objects with __del__ methods involved in cycles may not be collected
# Cleanup logic inside __del__ must be written carefully
# Because of this uncertainty, __del__ should not be used for critical resource management.

In [ ]:
import gc

class Cycle:
    def __init__(self, name):
        self.name = name
        self.ref = None
        print(f"{self.name} created")

    def __del__(self): # __del__ is a special method called when an object is about to be destroyed.
        print(f"{self.name} destroyed")

gc.disable() # Disable automatic garbage collection for demonstration purposes.

a = Cycle("A")
b = Cycle("B")

a.ref = b
b.ref = a
# Print the reference of b
print("References created")
print(b.ref)

del a
# Print the reference of b
print(b.ref)
del b

print("Deleted external references")

gc.collect() # Force a garbage collection cycle.
print("Garbage collection forced")

# Print the reference of b
print(b.ref)

In [13]:
# Practical Guidelines for Real Applications
# Garbage collection behavior explains several best practices:

# Avoid unnecessary circular references
# Be cautious when using __del__
# Use context managers for resource handling
# Do not rely on garbage collection timing for correctness
# In most applications, Python’s default garbage collection behavior is sufficient and should not be manually tuned.

In [14]:
# Exercise: Understanding Object Lifecycle with Cycles
# Create a Python script that demonstrates garbage collection behavior.

# Requirements:

# Define a class with a constructor and a destructor
# Create two objects that reference each other
# Remove all external references
# Force garbage collection using the gc module
# Observe when the destructor is called
# Objective:

# Understand the difference between reference counting cleanup and garbage collection cleanup
# Observe delayed destruction of cyclic objects
# The output should clearly show that object destruction happens only after garbage collection is triggered.

In [15]:
# Mutable vs immutable objects

# Mutable objects can be changed in place.
# Immutable objects cannot be changed once created.

# x = 10
# x = 20

# Immutable Objects in Python
# Common immutable types include:

# int
# float
# bool
# str
# tuple
# frozenset
# Once an immutable object is created, its value cannot be altered. Any operation that seems to modify it actually creates a new object.

a = 5
print(id(a))

a = a + 1
print(id(a))


4342431488
4342431520


In [16]:
# Mutable Objects in Python
# Common mutable types include:

# list
# dict
# set
# bytearray
# Most custom class instances
# Mutable objects allow modification of their internal state without changing their identity.

numbers = [1, 2, 3]
print(id(numbers))

numbers.append(4)
print(id(numbers))


4401417216
4401417216


In [17]:
# Reference Sharing and Mutability
# Mutability becomes important when multiple names reference the same object.

a = [10, 20]
b = a

b.append(30)

print(a)
print(b)


[10, 20, 30]
[10, 20, 30]


In [19]:
# Function Calls and Mutability
# Python passes object references to functions.

def add_item(data):
    data.append(100)

values = [1, 2, 3]
add_item(values)

print(values)


[1, 2, 3, 100]


In [20]:
# Immutable Objects in Function Calls

def increment(x):
    x = x + 1

n = 10
increment(n)

print(n)

10


In [21]:
# Mutability affects real applications in many ways:

# Shared configuration objects
# Caching logic
# Default function arguments
# Multithreaded data access
# API request/response manipulation
# Misunderstanding mutability often leads to:

# Unexpected side effects
# Data corruption
# Hard-to-debug production issues

In [22]:
# Common Pitfall: Mutable Default Arguments
# def add_user(user, users=[]):
#     users.append(user)
#     return users



# Correct Pattern for Default Arguments
# def add_user(user, users=None):
#     if users is None:
#         users = []
#     users.append(user)
#     return users

In [ ]:
def modify_list(lst):
    lst.append("changed")

def modify_number(num):
    num += 1

items = ["a", "b"]
value = 10

print("Before:", items, value)

modify_list(items)
modify_number(value)

print("After:", items, value)

In [23]:
# Exercise: Applying Mutability Concepts¶
# Create a Python script that demonstrates controlled mutation.

# Requirements:

# Create a function that receives a list and a number
# Modify the list inside the function
# Attempt to modify the number inside the function
# Print object identities before and after the function call
# Objective:

# Observe how mutable and immutable objects behave differently
# Verify identity changes using id()
# The output should clearly show that:

# Mutable objects retain identity while changing state
# Immutable objects change identity when modified

In [ ]:
# What Is the Global Interpreter Lock?
# The Global Interpreter Lock (GIL) is a mutex that protects access to Python objects, preventing multiple native threads from executing Python bytecodes at once. This means that even in a multi-threaded Python program, only one thread can execute Python code at a time.

# Understanding the GIL explains:
# Why multithreaded Python programs may not use multiple CPU cores effectively
# Why some threaded programs still perform well
# Why multiprocessing is often recommended for CPU-bound work

# Why Does the GIL Exist?
# The Global Interpreter Lock (GIL) exists primarily to simplify memory management in CPython, the reference implementation of Python. By ensuring that only one thread executes Python bytecode at a time, the GIL makes it easier to manage reference counts and prevent race conditions. This simplification comes at the cost of true parallelism in multi-threaded CPU-bound programs.

# The GIL exists to:

# Protect Python’s internal memory structures
# Make memory management fast and simple
# Avoid fine-grained locking across the entire interpreter



# What the GIL Actually Locks
# The GIL:
# Allows only one thread to execute Python bytecode at a time
# Does not prevent multiple threads from existing
# Does not block threads at the operating system level
# Threads still run concurrently, but only one thread can actively execute Python instructions at any given moment.


# What the GIL Does NOT Lock
# The GIL does not block:
# I/O operations (file I/O, network I/O, database calls)
# Native code execution inside C extensions (if they release the GIL)
# Multiple processes running in parallel
# This distinction is critical for understanding when threads are useful in Python.

import threading # threading module constructs higher-level threading interfaces on top of the lower level _thread module.

def task():
    for _ in range(5):
        print("Working")

t1 = threading.Thread(target=task)
t2 = threading.Thread(target=task)

t1.start()
t2.start()

# Even though two threads are created:
# They do not execute Python bytecode simultaneously
# Execution switches rapidly between threads
# Only one thread holds the GIL at any instant
# This switching can create the illusion of parallelism.

In [25]:
# CPU-Bound vs I/O-Bound Workloads

# CPU-bound tasks
# Heavy computation
# Number crunching
# Data processing loops

# For CPU-bound tasks:
# Threads compete for the GIL
# Only one thread makes progress at a time
# Multithreading does not scale across cores



# I/O-bound tasks
# Network calls
# File reads and writes
# Waiting for external systems

# For I/O-bound tasks:
# Threads release the GIL during waiting
# Other threads can run while one waits
# Multithreading can significantly improve throughput

In [26]:
# Demonstrating GIL Behavior with CPU-Bound Code

import threading
import time

def cpu_task():
    count = 0
    for _ in range(10_000_000):
        count += 1

start = time.time()

t1 = threading.Thread(target=cpu_task)
t2 = threading.Thread(target=cpu_task)

t1.start()
t2.start()

t1.join()
t2.join()

end = time.time()

print("Time taken:", end - start)

# Observation:
# Two threads do not halve execution time
# Threads take nearly the same time as a single-threaded version
# The GIL prevents true CPU parallelism

Time taken: 0.2406768798828125


In [27]:
# Demonstrating GIL Behavior with I/O-Bound Code

import threading
import time

def io_task():
    time.sleep(2)

start = time.time()

threads = []
for _ in range(5):
    t = threading.Thread(target=io_task)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

end = time.time()

print("Time taken:", end - start)

# Observation:
# Total time is close to the longest single sleep
# Threads overlap waiting time
# I/O-bound workloads benefit from threading
# Eg: If a program is executing each of thread at 0.5 seconds, total time will be around 0.5 seconds instead of 2.5 seconds

Time taken: 2.006499767303467


In [28]:
# How Python Switches Threads
# The GIL is periodically released:

# After a certain number of bytecode instructions
# When a thread performs blocking I/O
# This allows:

# Fair scheduling between threads
# Responsiveness in multithreaded applications
# The exact switching behavior is handled internally by the interpreter and does not need manual control in most applications.



In [29]:
# Real-World Implications

# Understanding the GIL explains common design choices:
# Why web servers use threading for I/O handling
# Why data processing pipelines use multiprocessing
# Why async programming is popular for high-concurrency I/O
# Why Python libraries offload heavy computation to C extensions

# The GIL is not a bug. It is a design trade-off.

In [30]:
# Threads are still useful when:
# The workload is I/O-bound
# Shared memory access is required
# Task coordination is complex
# Latency matters more than raw CPU throughput

# Examples:
# Web scraping
# Network services
# API orchestration
# Log processing

In [31]:
# Threads are a poor choice when:
# Work is CPU-heavy
# Computation dominates execution time
# Parallel CPU usage is required

# In these cases:
# Multiprocessing
# Vectorized libraries
# Native extensions are more appropriate.

In [32]:
# Exercise: Identifying GIL Impact
# Create two Python scripts.

# Script 1:

# Perform a CPU-heavy loop using two threads
# Measure total execution time
# Script 2:

# Perform a blocking I/O operation (such as sleep or network call) using multiple threads
# Measure total execution time
# Objective:

# Observe how threading behaves differently for CPU-bound and I/O-bound tasks
# Identify where the GIL limits performance and where it does not
# The output should clearly demonstrate that:

# CPU-bound threads do not scale across cores
# I/O-bound threads overlap effectively

In [33]:
# Collections module (deque, Counter, defaultdict, namedtuple) 

In [34]:
# What is a deque?
# A deque (double-ended queue) is a data structure provided by Python’s collections module.
# It allows fast insertion and removal of elements from both the left and right ends.
# Unlike lists, operations like adding or removing elements at the beginning are efficient in a deque.
# It is commonly used to implement queues, stacks, and sliding window algorithms.
# Internally, it is optimized for performance and thread-safe append and pop operations.

from collections import deque
# deque = Double Ended Queue (fast insertion/removal from both ends)

print("STEP 1: Create an empty deque")
dq = deque()
print(dq)
# OUTPUT: deque([])

print("-" * 50)


print("STEP 2: Append elements to the right")
dq.append(10)
print(dq)
# OUTPUT: deque([10])

dq.append(20)
print(dq)
# OUTPUT: deque([10, 20])

print("-" * 50)


print("STEP 3: Append elements to the left")
dq.appendleft(5)
print(dq)
# OUTPUT: deque([5, 10, 20])

dq.appendleft(1)
print(dq)
# OUTPUT: deque([1, 5, 10, 20])

print("-" * 50)


print("STEP 4: Remove element from the right (pop)")
removed = dq.pop()
print("Removed:", removed)
print(dq)
# OUTPUT:
# Removed: 20
# deque([1, 5, 10])

print("-" * 50)


print("STEP 5: Remove element from the left (popleft)")
removed = dq.popleft()
print("Removed:", removed)
print(dq)
# OUTPUT:
# Removed: 1
# deque([5, 10])

print("-" * 50)


print("STEP 6: Extend deque from the right")
dq.extend([30, 40, 50])
print(dq)
# OUTPUT: deque([5, 10, 30, 40, 50])

print("-" * 50)


print("STEP 7: Extend deque from the left")
dq.extendleft([100, 200])
print(dq)
# NOTE: extendleft inserts elements one-by-one from left
# OUTPUT: deque([200, 100, 5, 10, 30, 40, 50])

print("-" * 50)


print("STEP 8: Access elements using index")
print("First element:", dq[0])
# OUTPUT: First element: 200

print("Last element:", dq[-1])
# OUTPUT: Last element: 50

print("Current deque:", dq)
# OUTPUT: deque([200, 100, 5, 10, 30, 40, 50])

print("-" * 50)


print("STEP 9: Rotate deque")
dq.rotate(1)
print(dq)
# OUTPUT: deque([50, 200, 100, 5, 10, 30, 40])

dq.rotate(-2)
print(dq)
# OUTPUT: deque([100, 5, 10, 30, 40, 50, 200])

print("-" * 50)


print("STEP 10: Deque with maxlen (fixed size)")
limited_dq = deque(maxlen=3)
print(limited_dq)
# OUTPUT: deque([], maxlen=3)

limited_dq.append(10)
print(limited_dq)
# OUTPUT: deque([10], maxlen=3)

limited_dq.append(20)
print(limited_dq)
# OUTPUT: deque([10, 20], maxlen=3)

limited_dq.append(30)
print(limited_dq)
# OUTPUT: deque([10, 20, 30], maxlen=3)

limited_dq.append(40)
print(limited_dq)
# OUTPUT: deque([20, 30, 40], maxlen=3)
# NOTE: Oldest element (10) is automatically removed

print("-" * 50)


print("STEP 11: Clear the deque")
dq.clear()
print(dq)
# OUTPUT: deque([])


# # Important Pointers about deque:
# deque is much faster than list for queue-like operations
# appendleft() and popleft() are O(1) in deque
# list.insert(0, x) and list.pop(0) are slow
# maxlen makes deque perfect for sliding window problems
# rotate() helps in circular buffer use cases

STEP 1: Create an empty deque
deque([])
--------------------------------------------------
STEP 2: Append elements to the right
deque([10])
deque([10, 20])
--------------------------------------------------
STEP 3: Append elements to the left
deque([5, 10, 20])
deque([1, 5, 10, 20])
--------------------------------------------------
STEP 4: Remove element from the right (pop)
Removed: 20
deque([1, 5, 10])
--------------------------------------------------
STEP 5: Remove element from the left (popleft)
Removed: 1
deque([5, 10])
--------------------------------------------------
STEP 6: Extend deque from the right
deque([5, 10, 30, 40, 50])
--------------------------------------------------
STEP 7: Extend deque from the left
deque([200, 100, 5, 10, 30, 40, 50])
--------------------------------------------------
STEP 8: Access elements using index
First element: 200
Last element: 50
Current deque: deque([200, 100, 5, 10, 30, 40, 50])
--------------------------------------------------
STEP

In [35]:
# What Problem Does deque Solve?
# A Python list is efficient for appending and popping from the right side, but inefficient for operations at the left side.

# A deque (double-ended queue) is optimized for:
# Fast appends and pops from both ends
# Queue and stack–like behavior
# Sliding window problems

In [36]:
# Practical Example: Sliding Window History
from collections import deque

recent_logs = deque(maxlen=3)

recent_logs.append("login")
recent_logs.append("view_page")
recent_logs.append("logout")
recent_logs.append("login_again")

print(recent_logs)


deque(['view_page', 'logout', 'login_again'], maxlen=3)


In [37]:
# Exercise: deque Application
# Create a deque that stores the last 5 user actions.

# Add actions continuously
# Ensure only the most recent 5 actions are kept
# Print the deque after each insertion
# Objective:

# Understand bounded queues
# Observe automatic eviction behavior

In [38]:
# What Problem Does Counter Solve?
# A Counter is a specialized dictionary provided by Python’s collections module that counts the occurrences of hashable objects. It simplifies the task of tallying items in an iterable.
# It is commonly used for frequency analysis, histogram generation, and tallying votes or occurrences.

from collections import Counter

items = ["apple", "banana", "apple", "orange", "banana", "apple"]

counts = Counter(items)
print(counts)
# OUTPUT: Counter({'apple': 3, 'banana': 2, 'orange': 1})

Counter({'apple': 3, 'banana': 2, 'orange': 1})


In [39]:
# Practical Example: Log Level Analysis
from collections import Counter

logs = ["INFO", "ERROR", "INFO", "WARNING", "ERROR", "INFO"]

log_counts = Counter(logs)

print(log_counts["ERROR"])
print(log_counts.most_common(2))

2
[('INFO', 3), ('ERROR', 2)]


In [40]:
# Exercise: Counter Application
# Given a list of HTTP status codes:

# Count how many times each status appears
# Print the most frequent status code
# Objective:

# Replace manual dictionary counting with Counter
# Improve clarity and correctness

In [41]:
# defaultdict: Safe Defaults for Dictionaries
# What Problem Does defaultdict Solve?
# Accessing a missing key in a normal dictionary raises a KeyError.

# This often leads to patterns like:
# if key not in data:
#     data[key] = []
# defaultdict removes this boilerplate by providing default values automatically.

# Simple Example: Grouping Values
from collections import defaultdict

groups = defaultdict(list)

groups["admin"].append("Alice")
groups["admin"].append("Bob")
groups["user"].append("Charlie")

print(groups)

defaultdict(<class 'list'>, {'admin': ['Alice', 'Bob'], 'user': ['Charlie']})


In [ ]:
# Practical Example: Grouping Records by Category
from collections import defaultdict

orders = [
    ("electronics", 1000),
    ("books", 300),
    ("electronics", 500),
]

total_by_category = defaultdict(int)

for category, amount in orders:
    total_by_category[category] += amount

print(total_by_category)

defaultdict(<class 'int'>, {'electronics': 1500, 'books': 300})


In [43]:
# Exercise: defaultdict Application
# Create a defaultdict to group employees by department.

# Each department should map to a list of employee names
# Add multiple employees dynamically
# Objective:

# Eliminate manual key initialization
# Understand default factory behavior

In [44]:
# namedtuple: Lightweight Data Objects
# What Problem Does namedtuple Solve?
# Tuples are efficient but lack readability:

# user = ("Alice", 30, "admin")
# Index-based access reduces clarity and increases errors.

# namedtuple provides:

# Immutable objects
# Attribute-based access
# Low memory overhead

# Simple Example: Creating a namedtuple
from collections import namedtuple

User = namedtuple("User", ["name", "age", "role"])

u = User("Alice", 30, "admin")

print(u.name)
print(u.age)

Alice
30


In [45]:
# Practical Example: Structured Configuration Data
from collections import namedtuple

Config = namedtuple("Config", ["host", "port", "debug"])

config = Config("localhost", 8080, True)

print(config.host, config.port)

localhost 8080


In [46]:
from collections import namedtuple
# namedtuple creates tuple subclasses with named fields

print("STEP 1: Define a namedtuple")
Employee = namedtuple("Employee", ["id", "name", "salary"])
print(Employee)
# OUTPUT: <class '__main__.Employee'>

print("-" * 50)


print("STEP 2: Create an instance of namedtuple")
emp1 = Employee(101, "Alice", 75000)
print(emp1)
# OUTPUT: Employee(id=101, name='Alice', salary=75000)

print("-" * 50)


print("STEP 3: Access values using index (tuple behavior)")
print(emp1[0])
# OUTPUT: 101

print(emp1[1])
# OUTPUT: Alice

print("-" * 50)


print("STEP 4: Access values using field names (main advantage)")
print(emp1.id)
# OUTPUT: 101

print(emp1.name)
# OUTPUT: Alice

print(emp1.salary)
# OUTPUT: 75000

print("-" * 50)


print("STEP 5: namedtuple is immutable (cannot modify)")
# emp1.salary = 80000
# OUTPUT (if uncommented):
# AttributeError: can't set attribute

print("-" * 50)


print("STEP 6: Using _replace() to create a modified copy")
emp2 = emp1._replace(salary=80000)
print(emp2)
# OUTPUT: Employee(id=101, name='Alice', salary=80000)

print("Original object remains unchanged:", emp1)
# OUTPUT: Employee(id=101, name='Alice', salary=75000)

print("-" * 50)


print("STEP 7: Convert namedtuple to dictionary")
emp_dict = emp1._asdict()
print(emp_dict)
# OUTPUT: {'id': 101, 'name': 'Alice', 'salary': 75000}

print("-" * 50)


print("STEP 8: Create namedtuple with default values")
Student = namedtuple("Student", ["roll", "name", "marks"], defaults=[0])
print(Student)
# OUTPUT: <class '__main__.Student'>

student1 = Student(1, "Rahul")
print(student1)
# OUTPUT: Student(roll=1, name='Rahul', marks=0)

print("-" * 50)


print("STEP 9: Access field names and defaults metadata")
print(Student._fields)
# OUTPUT: ('roll', 'name', 'marks')

print(Student._field_defaults)
# OUTPUT: {'marks': 0}

print("-" * 50)


print("STEP 10: namedtuple behaves like a tuple")
print(len(emp1))
# OUTPUT: 3

print(tuple(emp1))
# OUTPUT: (101, 'Alice', 75000)


STEP 1: Define a namedtuple
<class '__main__.Employee'>
--------------------------------------------------
STEP 2: Create an instance of namedtuple
Employee(id=101, name='Alice', salary=75000)
--------------------------------------------------
STEP 3: Access values using index (tuple behavior)
101
Alice
--------------------------------------------------
STEP 4: Access values using field names (main advantage)
101
Alice
75000
--------------------------------------------------
STEP 5: namedtuple is immutable (cannot modify)
--------------------------------------------------
STEP 6: Using _replace() to create a modified copy
Employee(id=101, name='Alice', salary=80000)
Original object remains unchanged: Employee(id=101, name='Alice', salary=75000)
--------------------------------------------------
STEP 7: Convert namedtuple to dictionary
{'id': 101, 'name': 'Alice', 'salary': 75000}
--------------------------------------------------
STEP 8: Create namedtuple with default values
<class '__

In [ ]:
# Exercise: namedtuple Application
# Define a namedtuple to represent a product with:

# id
# name
# price
# Create multiple product instances and print their attributes.

# Objective:

# Replace positional tuples with self-documenting structures

In [47]:
# Itertools and Generators for Lazy Evaluation
# Why Lazy Evaluation Matters

# In many real-world applications, data is:
# Large
# Continuous
# Expensive to compute
# Not fully needed at once

# Creating all data upfront wastes:
# Memory
# CPU time
# I/O bandwidth

# Lazy evaluation solves this by producing values only when they are needed, not before.

# Python supports lazy evaluation primarily through:
# Generators
# The itertools module

In [48]:
# What is itertools?

# itertools is a Python module that provides fast, memory-efficient tools for working with iterators.
# It helps generate values on demand instead of storing everything in memory.
# Most tools are used for looping, combinations, filtering, and grouping data.
# It is especially useful when working with large datasets or infinite sequences.
# Internally, it avoids creating intermediate lists, making programs faster and lighter.

import itertools
# itertools provides iterator building blocks for efficient looping

print("STEP 1: count() – Infinite counter")
counter = itertools.count(start=1, step=1) # start=1, step=1 means it will start counting from 1 and increment by 1 each time.

print(next(counter)) # Get the next value from the counter
# OUTPUT: 1

print(next(counter)) # Get the next value from the counter
# OUTPUT: 2

print(next(counter)) # Get the next value from the counter
# OUTPUT: 3

print("-" * 50)


print("STEP 2: islice() – Limit an infinite iterator")
limited = itertools.islice(counter, 3) # Take only the next 3 values from the infinite counter

print(list(limited)) # Convert the limited iterator to a list and print it
# OUTPUT: [4, 5, 6]

print("-" * 50)


print("STEP 3: repeat() – Repeat a value fixed number of times")
rep = itertools.repeat("A", 4) # Repeat "A" 4 times

print(list(rep)) # Convert the repeated iterator to a list and print it
# OUTPUT: ['A', 'A', 'A', 'A']

print("-" * 50)


print("STEP 4: cycle() – Cycle through elements endlessly")
cycler = itertools.cycle([1, 2, 3]) # Cycle through the list [1, 2, 3] endlessly

print(next(cycler)) # Get the next value from the cycler
# OUTPUT: 1

print(next(cycler)) # Get the next value from the cycler
# OUTPUT: 2

print(next(cycler)) # Get the next value from the cycler
# OUTPUT: 3

print(next(cycler)) # Get the next value from the cycler
# OUTPUT: 1

print("-" * 50)


print("STEP 5: chain() – Combine multiple iterables")
combined = itertools.chain([1, 2], ['a', 'b'], [True, False]) # Combine multiple iterables into one

print(list(combined)) # Convert the combined iterator to a list and print it
# OUTPUT: [1, 2, 'a', 'b', True, False]

print("-" * 50)


print("STEP 6: accumulate() – Running total (prefix sum)")
nums = [1, 2, 3, 4]
acc = itertools.accumulate(nums)

print(list(acc)) # Convert the accumulated iterator to a list and print it
# OUTPUT: [1, 3, 6, 10]

print("-" * 50)


print("STEP 7: filterfalse() – Keep elements where condition is False")
nums = [1, 2, 3, 4, 5]
filtered = itertools.filterfalse(lambda x: x % 2 == 0, nums)

print(list(filtered)) # Convert the filtered iterator to a list and print it
# OUTPUT: [1, 3, 5]

print("-" * 50)


print("STEP 8: dropwhile() – Drop elements while condition is True")
nums = [1, 2, 3, 4, 1, 2]
dropped = itertools.dropwhile(lambda x: x < 4, nums) # Drop elements while they are less than 4

print(list(dropped)) # Convert the dropped iterator to a list and print it
# OUTPUT: [4, 1, 2]

print("-" * 50)


print("STEP 9: takewhile() – Take elements while condition is True")
taken = itertools.takewhile(lambda x: x < 4, nums) # Take elements while they are less than 4

print(list(taken)) # Convert the taken iterator to a list and print it
# OUTPUT: [1, 2, 3]

print("-" * 50)


print("STEP 10: combinations() – Unique combinations (order doesn't matter)")
comb = itertools.combinations([1, 2, 3], 2) # Generate unique combinations of length 2

print(list(comb)) # Convert the combinations iterator to a list and print it
# OUTPUT: [(1, 2), (1, 3), (2, 3)]

print("-" * 50)


print("STEP 11: permutations() – All orderings")
perm = itertools.permutations([1, 2, 3], 2) # Generate all permutations of length 2

print(list(perm))
# OUTPUT: [(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)]

print("-" * 50)


print("STEP 12: product() – Cartesian product")
prod = itertools.product([1, 2], ['a', 'b']) # Generate the Cartesian product of the two lists

print(list(prod)) # Convert the product iterator to a list and print it
# OUTPUT: [(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b')]

print("-" * 50)


print("STEP 13: groupby() – Group consecutive elements")
data = [1, 1, 2, 2, 2, 3, 1]

grouped = itertools.groupby(data)

for key, group in grouped:
    print(key, list(group))

# OUTPUT:
# 1 [1, 1]
# 2 [2, 2, 2]
# 3 [3]
# 1 [1]


STEP 1: count() – Infinite counter
1
2
3
--------------------------------------------------
STEP 2: islice() – Limit an infinite iterator
[4, 5, 6]
--------------------------------------------------
STEP 3: repeat() – Repeat a value fixed number of times
['A', 'A', 'A', 'A']
--------------------------------------------------
STEP 4: cycle() – Cycle through elements endlessly
1
2
3
1
--------------------------------------------------
STEP 5: chain() – Combine multiple iterables
[1, 2, 'a', 'b', True, False]
--------------------------------------------------
STEP 6: accumulate() – Running total (prefix sum)
[1, 3, 6, 10]
--------------------------------------------------
STEP 7: filterfalse() – Keep elements where condition is False
[1, 3, 5]
--------------------------------------------------
STEP 8: dropwhile() – Drop elements while condition is True
[4, 1, 2]
--------------------------------------------------
STEP 9: takewhile() – Take elements while condition is True
[1, 2, 3]
-------

In [ ]:
# Eager vs Lazy Evaluation

# Eager Evaluation
# All values are computed immediately
# Stored fully in memory
# Common with lists

# Example:
# numbers = [x * 2 for x in range(10)]
# All values are created and stored at once.



# Lazy Evaluation
# Values are computed one at a time
# Generated only when requested
# Memory usage stays low

# Example:
# numbers = (x * 2 for x in range(10))
# No values are created upfront.

Generators in Python
--

Generators are functions that return traversable objects.

They produce items one at a time and only when required.

__Generators are run along with for loop.__

__Advantages:__

1. Easy to implement

2. Better Memory Management and Utilization.

3. Can be used to produce infinite terms.

4. Can be used to pipeline a number of operations.

__Normal Functions vs Generators:__

1.
Generator Functions: Make use of 'yield' function.
Normal Functions: Make use of 'return' keyword.

2.
Generator Functions: Run when next() method is called.
Normal Functions: Run when name of the method is called.

3.
Generator Functions: Produce items one at a time and only when required.
Normal Functions: Produce all items at once.

In [49]:
def abc():
    return "Hi"

for i in range(4):
    print(abc())

Hi
Hi
Hi
Hi


In [50]:
# Generator-Function
def abc(): 
    yield 1            
    yield 2            
    yield 3            
for i in abc():
    print(i)

1
2
3


In [51]:
# Generator-Object
def abc(): 
    yield 1
    yield 2
    yield 3
   
# x is a generator object 
x = abc() 
  
# Iterating over the generator object using next 
print(x.__next__()) 

1


In [52]:
print("Hello World")

Hello World


In [53]:
print(x.__next__())

2


In [54]:
print(x.__next__())

3


In [55]:
print("Hiiiiiii")

Hiiiiiii


In [56]:
print(x.__next__())

StopIteration: 

In [57]:
# So a generator function returns an generator object that is iterable, i.e., can be used as an Iterators 

In [58]:
# Snippet 2:
def xyz(d):
    for x in d.items():
        yield x
dict = {1: "Lets", 2: "Upgrade"}
b=xyz(dict)
print(b)
b.__next__()

<generator object xyz at 0x106a15700>


(1, 'Lets')

In [59]:
b.__next__()

(2, 'Upgrade')

In [60]:
b.__next__()

StopIteration: 

In [61]:
# Snippet 3:
def mno(i):
    while i<=3:
        yield i
        i=i+1
j = mno(2)
print(j.__next__()) # This will print one by one
# for i in j:
#     print(i)

2


In [62]:
print(j.__next__())

3


In [63]:
print(j.__next__())

StopIteration: 

In [64]:
# Use Case 1: Generator for Fibonacci Numbers.
def fib(limit):       
    # Initialize first two Fibonacci Numbers  
    a, b = 0, 1
  
    # One by one yield next Fibonacci Number 
    while a < limit: 
        yield a 
        a = b
        b = a+b
  
# Create a generator object 
x = fib(20) 

In [65]:
# Iterating over the generator object using next 
print(x.__next__()) 

0


In [66]:
# Iterating over the generator object using next 
print(x.__next__()) 

1


In [67]:
# Iterating over the generator object using next 
print(x.__next__()) 

2


In [68]:
# Iterating over the generator object using next 
print(x.__next__()) 

4


In [69]:
# Iterating over the generator object using next 
print(x.__next__()) 

8


In [70]:
# Iterating over the generator object using for 
# in loop. 
print("\nUsing for in loop") 
for i in x:  
    print(i)


Using for in loop
16


__Practical Processing:__

1. It is used in handling large data files such as log files. 


2. Generators provide a space efficient method for such data processing as only parts of the file are handled at one given point in time. 


3. We can also use Iterators for these purposes, but Generator provides a quick way


4. Represent Infinite Stream: Generators are excellent medium to represent an infinite stream of data. Infinite streams cannot be stored in memory and since generators produce only one item at a time, it can represent infinite stream of data.

In [71]:
# Generator vs List in Memory Behavior
# numbers_list = [x for x in range(1_000_000)]
# numbers_gen = (x for x in range(1_000_000))


In [72]:
# Chaining Generators
data = range(10)

filtered = (x for x in data if x % 2 == 0)
squared = (x * x for x in filtered)

for value in squared:
    print(value)

0
4
16
36
64


In [73]:
# def read_lines(file_path):
#     with open(file_path) as f:
#         for line in f:
#             yield line.strip()

# for line in read_lines("sample.txt"):
#     print(line)

In [74]:
# Exercise: Building a Lazy Data Pipeline
# Create a Python script that:

# Generates numbers from 1 to infinity
# Filters only numbers divisible by 3
# Squares each filtered number
# Prints only the first 5 results
# Requirements:

# Use generators or itertools
# Do not create intermediate lists
# Stop execution cleanly
# Objective:

# Apply lazy evaluation concepts
# Build a memory-efficient pipeline
# Control infinite iteration safely


In [ ]:
# Comprehensions and generator expressions – performance considerations 

# Comprehensions and Generator Expressions – Performance Considerations

## Why Performance Matters in Everyday Python Code

Comprehensions and generator expressions are often introduced as *syntactic shortcuts*. In reality, they are also **performance tools** when used correctly.

Understanding how they behave helps:

* Reduce memory usage
* Improve execution speed
* Write clearer, more maintainable code
* Avoid hidden performance costs

This topic focuses on **practical performance awareness**, not micro-optimizations.

---

## What Are Comprehensions?

A **comprehension** is a compact way to create a new collection by transforming and optionally filtering data.

Common types:

* List comprehension
* Set comprehension
* Dictionary comprehension

Example:

```python
squares = [x * x for x in range(10)]
```

This creates a **fully materialized list** in memory.

---

## What Are Generator Expressions?

A **generator expression** looks similar to a list comprehension but produces values lazily.

Example:

```python
squares = (x * x for x in range(10))
```

This does **not** create all values upfront. Values are generated only when requested.

---

## Visual Difference: Memory Behavior

```python
numbers_list = [x for x in range(1_000_000)]
numbers_gen = (x for x in range(1_000_000))
```

Key differences:

* The list stores all one million elements in memory
* The generator stores only the generation logic
* Memory usage differs drastically

This distinction is critical when working with large datasets.

---

## Execution Timing: When Work Actually Happens

### List Comprehension

```python
data = [x * 2 for x in range(5)]
```

Behavior:

* All computations happen immediately
* Execution completes before assignment finishes

### Generator Expression

```python
data = (x * 2 for x in range(5))
```

Behavior:

* No computation happens at creation time
* Computation happens during iteration

This affects both performance and program flow.

---

## Simple Timing Comparison (Conceptual)

```python
import time

start = time.time()
data = [x * 2 for x in range(10_000_000)]
end = time.time()

print("List comprehension time:", end - start)
```

Compared to:

```python
start = time.time()
data = (x * 2 for x in range(10_000_000))
end = time.time()

print("Generator creation time:", end - start)
```

Explanation:

* List comprehension time includes computation
* Generator creation time does not
* Actual computation for generators happens later

---

## When List Comprehensions Are Faster

List comprehensions can be faster when:

* The dataset is small or medium-sized
* The result is needed multiple times
* Random access is required
* The data must be reused

Example:

```python
values = [x * 2 for x in range(1000)]
total = sum(values)
max_value = max(values)
```

Using a generator here would recompute values or require conversion.

---

## When Generator Expressions Are Better

Generator expressions are better when:

* Data is large or infinite
* Values are consumed only once
* Data is processed in a pipeline
* Memory efficiency is important

Example:

```python
total = sum(x * 2 for x in range(10_000_000))
```

No intermediate list is created.

---

## Comprehensions vs Traditional Loops

```python
result = []
for x in range(10):
    result.append(x * x)
```

Compared to:

```python
result = [x * x for x in range(10)]
```

List comprehensions are:

* More concise
* Often faster due to internal optimizations
* Easier to read when logic is simple

However, readability should not be sacrificed for brevity.

---

## Set and Dictionary Comprehensions

Set comprehension:

```python
unique_lengths = {len(word) for word in ["cat", "dog", "elephant"]}
```

Dictionary comprehension:

```python
length_map = {word: len(word) for word in ["cat", "dog", "elephant"]}
```

These are often clearer and faster than manual loops.

---

## Nested Comprehensions: Use With Caution

```python
pairs = [(x, y) for x in range(3) for y in range(3)]
```

While valid, nested comprehensions:

* Can reduce readability
* Can hide performance costs
* Should be avoided if logic becomes complex

If the logic is not immediately clear, a loop is often better.

---

## Generator Exhaustion and Reuse

```python
gen = (x * 2 for x in range(3))

print(list(gen))
print(list(gen))
```

Behavior:

* The first conversion consumes the generator
* The second produces an empty list

Generators are single-use by design. This must be considered when choosing between generators and lists.

---

## Script-Based Performance Comparison

This code must be saved as `comprehension_vs_generator.py` and executed from the terminal using:

```
python comprehension_vs_generator.py
```

It should not be run inside a Jupyter Notebook.

```python
import time

N = 5_000_000

start = time.time()
lst = [x * 2 for x in range(N)]
print("List comprehension:", time.time() - start)

start = time.time()
gen = (x * 2 for x in range(N))
total = sum(gen)
print("Generator expression:", time.time() - start)
```

Observation:

* List comprehensions use more memory
* Generator expressions compute values on demand
* Performance depends on usage pattern, not syntax alone

---

## Real-World Perspective

In real applications:

* Use list comprehensions when you need the data structure
* Use generator expressions for streaming, aggregation, and pipelines
* Combine generators with `sum`, `any`, `all`, `min`, `max`
* Prefer clarity over cleverness

Performance improvements should be driven by measurement, not assumptions.

---

## Exercise: Choosing the Right Tool

Create two implementations for processing numbers from 1 to 1,000,000.

Implementation 1:

* Use a list comprehension
* Store all squared values
* Compute the sum

Implementation 2:

* Use a generator expression
* Compute the sum directly without storing values

Compare:

* Execution time
* Memory behavior (conceptually)

Objective:

* Decide when eager evaluation is appropriate
* Decide when lazy evaluation is more efficient

---

In [75]:
# Module 3: Context Managers and Decorators
# • Writing custom context managers with contextlib
# • Chaining decorators and parameterized decorators

In [76]:
# Writing custom context managers with contextlib

# Why Context Managers Exist
# Many operations in Python require setup and cleanup logic:
# - Opening and closing files
# - Acquiring and releasing locks
# - Opening and closing database connections
# - Measuring execution time
# - Temporarily changing application state
# If cleanup is forgotten or skipped due to an exception, resources leak and bugs appear.

# Context managers provide a reliable, structured way to ensure cleanup always happens.

# Consider manual resource handling:
# file = open("data.txt", "r")
# data = file.read()
# file.close()

# Using try/finally improves safety but increases boilerplate and repetition.

# The with Statement
# The with statement solves this problem by guaranteeing cleanup.
# with open("data.txt", "r") as file:
#     data = file.read()



# How Context Managers Work Internally
# A context manager is any object that defines two special methods:
# a. __enter__()
# b. __exit__()

# Execution flow:
# __enter__() is called at the start of the with block
# The block executes
# __exit__() is called at the end, regardless of success or failure


In [77]:
# # Writing a Custom Context Manager Using a Class

# # Simple Example: Logging Resource Usage
# class ResourceLogger:
#     def __enter__(self): # __enter__ is called at the start of the with block
#         print("Resource acquired")
#         return self # Optional: return a resource object. self is returned here.

#     def __exit__(self, exc_type, exc_value, traceback): # __exit__ is called at the end of the with block. self - the instance of the class, exc_type - exception type, exc_value - exception value, traceback - traceback object
#         print("Resource released")

# Usage:
# with ResourceLogger():
#     print("Using the resource")
# Explanation:

# __enter__() runs before the block
# __exit__() runs after the block
# Cleanup always executes

In [78]:
# Handling Exceptions in __exit__
# The __exit__() method receives:
# exc_type: exception class (or None)
# exc_value: exception instance (or None)
# traceback: traceback object (or None)

# class ExceptionAwareContext:
#     def __enter__(self):
#         print("Start")
#         return self

#     def __exit__(self, exc_type, exc_value, traceback):
#         if exc_type:
#             print("An exception occurred")
#         print("End")

In [79]:
# Why Class-Based Context Managers Can Feel Heavy

# Class-based context managers are powerful, but:
# They require boilerplate
# They may feel verbose for simple use cases
# Many scenarios only need setup and cleanup logic
# This is where contextlib becomes useful.

In [80]:
# The contextlib module provides utilities to:
# Create context managers more easily
# Reduce boilerplate
# Improve readability

# The most commonly used tool is the @contextmanager decorator.

In [81]:
# Writing a Context Manager Using @contextmanager

# The @contextmanager decorator allows writing a context manager using a generator-like function.

# from contextlib import contextmanager

# @contextmanager
# def simple_context():
#     print("Enter")
#     yield
#     print("Exit")

# Usage:
# with simple_context():
#     print("Inside block")

In [ ]:
# Yielding Values from a Context Manager
from contextlib import contextmanager

@contextmanager
def managed_resource(name):
    print(f"Acquiring {name}")
    resource = f"Resource({name})"
    yield resource
    print(f"Releasing {name}")

# Usage:
with managed_resource("DB") as res:
    print(res)

# The value after yield is what as res receives
# yield separates entry logic and exit logic
# Cleanup code always runs after the block finishes

# Very useful for managing files, database connections, locks, and timers

Acquiring DB
Resource(DB)
Releasing DB


In [ ]:
# Handling Exceptions with contextlib
from contextlib import contextmanager

@contextmanager
def safe_context():
    print("Start")
    try:
        yield
    except Exception as e:
        print("Exception handled:", e)
        raise
    finally:
        print("Cleanup")


with safe_context():
    print("Inside block")
    raise ValueError("Something went wrong")


# Exceptions raised inside with are caught at the yield point
# finally guarantees cleanup
# Re-raising preserves original error behavior
# Very useful for database transactions, file locks, and rollback logic

Start
Inside block
Exception handled: Something went wrong
Cleanup


ValueError: Something went wrong

In [87]:
# Practical Example: Timing Code Execution
from contextlib import contextmanager
import time

@contextmanager
def timer():
    start = time.time()
    yield
    end = time.time()
    print("Elapsed time:", end - start)
# Usage:
with timer():
    time.sleep(1)

Elapsed time: 1.0050382614135742


In [88]:
# When to Use Class-Based vs contextlib
# Use class-based context managers when:
# You need to store state across multiple methods
# The context manager is complex
# Reusability and extensibility matter

# Use contextlib when:
# Setup and cleanup logic is simple
# Readability is more important than structure
# You want minimal boilerplate
# Both approaches are valid and complementary.



In [89]:
# Exercise: Creating a Custom Context Manager
# Create a custom context manager using contextlib.

# Requirements:

# Print a message when entering the context
# Yield a value that can be used inside the with block
# Print a message when exiting the context
# Ensure cleanup happens even if an exception occurs
# Example use case ideas:

# Temporarily changing a configuration value
# Measuring execution time
# Simulating resource acquisition and release
# Objective:

# Apply @contextmanager correctly
# Understand how yield splits enter and exit logic

In [90]:
# What is a decorator?
# A decorator is a way to add extra behavior to a function without editing the function’s original code.

# Why do we need decorators?
# Without decorators, you would repeat the same code again and again:

# Example: If you want timing for 10 functions, you’d manually add timing code inside all 10 → messy + duplicate code.
# With decorators:
# You write timing logic once
# Apply it to any function with @timer
# Your actual function stays clean

In [91]:
# A normal function

def greet():
    print("Hello!")

greet()
# OUTPUT: Hello!


Hello!


In [92]:
# Functions can be stored in variables

def greet():
    print("Hello!")

x = greet          # No brackets. We are storing the function itself.
x()                # Now call it using x
# OUTPUT: Hello!


Hello!


In [ ]:
# A function can accept another function as input

def greet():
    print("Hello!")

def caller(func):          # func will receive a function
    func()                 # calling that function

caller(greet)
# OUTPUT: Hello!


# A function can take another function”
# This is the base of decorators.

Hello!


In [ ]:
# Add extra behavior around a function (manual wrapping)
# Let’s say we want:
# print “Start”
# run greet()
# print “End”

def greet():
    print("Hello!")

def add_extra_behavior(func):
    print("Start")
    func()
    print("End")

add_extra_behavior(greet)
# OUTPUT:
# Start
# Hello!
# End


# But there’s a problem:
# This does not create a new function
# It just runs it once
# A decorator should create a new enhanced function.

Start
Hello!
End


In [96]:
# Create a wrapper function (this is the decorator style)

def greet():
    print("Hello!")

def decorator(func):
    # wrapper is a new function that adds extra behavior
    def wrapper():
        print("Start")
        func()             # calling the original function
        print("End")
    return wrapper         # return the new enhanced function

new_greet = decorator(greet)   # decorate greet manually
new_greet()
# OUTPUT:
# Start
# Hello!
# End


# So, decorator = function that takes a function and returns a new function

Start
Hello!
End


In [97]:
# Python gives a shortcut syntax: @decorator

# Instead of writing:
# new_greet = decorator(greet)
# We can write:
# @decorator
# def greet():
#     print("Hello!")


In [98]:
# Full Code:

def decorator(func):
    def wrapper():
        print("Start")
        func()
        print("End")
    return wrapper

@decorator
def greet():
    print("Hello!")

greet()
# OUTPUT:
# Start
# Hello!
# End
# The @decorator syntax is just shorthand for:
# greet = decorator(greet)

Start
Hello!
End


In [99]:
# Decorator for functions with arguments
# Most real functions have inputs.

def decorator(func):
    def wrapper(name):          # wrapper accepts the same argument
        print("Start")
        func(name)              # pass it to original function
        print("End")
    return wrapper

@decorator
def greet(name):
    print("Hello", name)

greet("Darshan")
# OUTPUT:
# Start
# Hello Darshan
# End

# But this works only if the function has exactly one argument.
# We want a generic solution.

Start
Hello Darshan
End


In [101]:
# Generic decorator using *args and **kwargs
# This is the real production version.

def decorator(func):
    def wrapper(*args, **kwargs):      # accepts any arguments
        print("Start")
        result = func(*args, **kwargs) # call original function with same args
        print("End")
        return result                  # return original result if any
    return wrapper

@decorator
def add(a, b):
    return a + b

print(add(10, 20))
# OUTPUT:
# Start
# End
# 30


Start
End
30


In [102]:
# Conclusion Notes:

# A decorator is a function that:
# takes a function as input
# adds extra behavior
# returns a new function

# @decorator is just a shortcut for:
# func = decorator(func)

# Use *args, **kwargs to support any function signature.

In [103]:
# Chaining decorators (@d1 then @d2)
# Chaining decorators means applying more than one decorator to the same function.

# Why do we need chaining?
# Because in real programs, a function may need:
# logging
# timing
# authentication
# validation

# Instead of writing one big decorator, we:
# write small, focused decorators
# combine them when needed

# This keeps code clean, reusable, and readable



# The Most Important Rule (Must Remember)
# @d1
# @d2
# def func():
#     pass
# This means:
# func = d1(d2(func))
# Decorators are applied from bottom to top.

In [104]:
# Step 1: Define the first decorator
# This decorator prints messages before and after a function.

def decorator_one(func):
    def wrapper():
        print("Decorator ONE - Start")
        func()
        print("Decorator ONE - End")
    return wrapper


In [105]:
# Step 2: Define the second decorator
# This decorator does a similar thing but with different messages.

def decorator_two(func):
    def wrapper():
        print("Decorator TWO - Start")
        func()
        print("Decorator TWO - End")
    return wrapper


In [ ]:
# Step 3: Apply both decorators to one function (chaining)

@decorator_one
@decorator_two
def greet():
    print("Hello!")

# This is equivalent to:
# greet = decorator_one(decorator_two(greet))

In [107]:
# Step 4: Call the function
greet()
# OUTPUT:
# Decorator ONE - Start
# Decorator TWO - Start
# Hello!
# Decorator TWO - End
# Decorator ONE - End

Decorator ONE - Start
Decorator TWO - Start
Hello!
Decorator TWO - End
Decorator ONE - End


In [109]:
# Step 7: Chaining with arguments (real-world safe version)

def decorator_one(func):
    def wrapper(*args, **kwargs):
        print("Decorator ONE - Start")
        result = func(*args, **kwargs)
        print("Decorator ONE - End")
        return result
    return wrapper


def decorator_two(func):
    def wrapper(*args, **kwargs):
        print("Decorator TWO - Start")
        result = func(*args, **kwargs)
        print("Decorator TWO - End")
        return result
    return wrapper


@decorator_one
@decorator_two
def add(a, b):
    return a + b


print(add(10, 20))


Decorator ONE - Start
Decorator TWO - Start
Decorator TWO - End
Decorator ONE - End
30


In [110]:
# Most common confusuons:

# Confusion 1: Which decorator runs first?
# - Bottom decorator executes first
# - Top decorator executes last

# Confusion 2: Does the function run multiple times?
# No.
# The function runs once
# Decorators just surround it

# Confusion 3: Can decorators change return values?
# Yes.
# They can modify, replace, or log return values


# Conclusion:
# - Chaining decorators means applying multiple decorators to one function
# - Decorators are applied bottom to top
# - Execution happens like nested wrappers
# - Each decorator should do one clean job
# - This pattern is heavily used in real frameworks (logging, auth, retries)

In [111]:
# What is a parameterized decorator?
# A parameterized decorator is a decorator that accepts arguments.
# Example:
# @retry(times=3)

# Why do we need parameterized decorators?
# Because behavior often needs configuration:
# Retry count
# Timeout value
# Logging level
# Cache size
# Access role
# Instead of hardcoding values, we pass them as arguments.

# Key Rule:
# @retry(times=3)
# def func():
#     pass
# Is equivalent to:
# func = retry(times=3)(func)
# So there are three layers involved.

In [112]:
# Step 1: Normal decorator

def decorator(func):
    def wrapper():
        print("Before")
        func()
        print("After")
    return wrapper


In [113]:
# Step 2: Why this is NOT enough
# We want:
# @retry(times=3)

# But this won’t work with a normal decorator because:
# - retry would immediately receive a function
# But here it receives times=3
# So we need one extra function layer.

In [114]:
# Step 3: Structure of a parameterized decorator

def retry(times):          # 1) receives arguments
    def decorator(func):   # 2) receives function
        def wrapper():     # 3) wraps function
            pass
        return wrapper
    return decorator


In [115]:
# Step 4: Implement a simple retry decorator

def retry(times):
    def decorator(func):
        def wrapper():
            for attempt in range(1, times + 1):
                try:
                    print(f"Attempt {attempt}")
                    return func()
                except Exception as e:
                    print("Error:", e)
            print("All retries failed")
        return wrapper
    return decorator


In [116]:
# Step 5: Apply the decorator

@retry(times=3)
def risky_task():
    print("Running risky task")
    raise ValueError("Something went wrong")



In [117]:
# Step 6: Call the function
risky_task()

Attempt 1
Running risky task
Error: Something went wrong
Attempt 2
Running risky task
Error: Something went wrong
Attempt 3
Running risky task
Error: Something went wrong
All retries failed


In [119]:
# Step 7: Support arguments in decorated function (production-safe)

def retry(times):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for attempt in range(1, times + 1):
                try:
                    print(f"Attempt {attempt}")
                    return func(*args, **kwargs)
                except Exception as e:
                    print("Error:", e)
            print("All retries failed")
        return wrapper
    return decorator


In [120]:
# Step 8: Apply to a function with parameters

@retry(times=2)
def divide(a, b):
    return a / b

print(divide(10, 0))


Attempt 1
Error: division by zero
Attempt 2
Error: division by zero
All retries failed
None


In [121]:
# Mistakes to avoid:
# 1. Forgetting the extra function layer
# 2. Writing @retry instead of @retry()
# 3. Not using *args, **kwargs
# 4. Forgetting to return the wrapper

# Decorators: From Basics to Chaining and Parameterized Decorators

## What Is a Decorator?

In Python, **functions are objects**. This means:

* A function can be assigned to a variable
* A function can be passed as an argument
* A function can be returned from another function

A **decorator** is simply a function that:

* Takes another function as input
* Adds some extra behavior
* Returns a new function

The original function’s code is not modified, but its behavior is extended.

---

## Why Decorators Are Needed

In real programs, certain behaviors are required repeatedly across many functions:

* Logging when a function is called
* Checking permissions
* Measuring execution time
* Validating inputs

Without decorators, this logic would be duplicated inside every function, leading to:

* Repetitive code
* Harder maintenance
* Higher chance of errors

Decorators allow separating **core business logic** from **cross-cutting concerns**.

---

## Functions as Objects (Foundation Concept)

```python
def greet():
    print("Hello")

say_hello = greet
say_hello()
```

Explanation:

* `greet` is a function object
* `say_hello` now refers to the same function
* Calling `say_hello()` executes the function

This behavior makes decorators possible.

---

## A Function That Wraps Another Function

```python
def my_decorator(func):
    def wrapper():
        print("Before function call")
        func()
        print("After function call")
    return wrapper
```

Explanation:

* `my_decorator` receives a function
* `wrapper` adds behavior before and after the function
* `wrapper` is returned as a new function

---

## Applying a Decorator Manually

```python
def greet():
    print("Hello")

greet = my_decorator(greet)
greet()
```

Execution flow:

1. `my_decorator(greet)` returns `wrapper`
2. `greet` now refers to `wrapper`
3. Calling `greet()` executes the wrapped logic

This manual replacement is exactly what decorators automate.

---

## The `@` Syntax for Decorators

Python provides syntactic sugar to apply decorators cleanly.

```python
@my_decorator
def greet():
    print("Hello")
```

This is equivalent to:

```python
greet = my_decorator(greet)
```

The `@` syntax improves readability and is the standard way decorators are written.

---

## Decorators with Function Arguments

Most real functions accept arguments, so decorators must handle them.

```python
def log_decorator(func):
    def wrapper(*args, **kwargs):
        print("Function called")
        return func(*args, **kwargs)
    return wrapper
```

Explanation:

* `*args` captures positional arguments
* `**kwargs` captures keyword arguments
* The decorator works for any function signature

---

## Simple Practical Example: Logging Decorator

```python
def log_call(func):
    def wrapper(*args, **kwargs):
        print(f"Calling function: {func.__name__}")
        return func(*args, **kwargs)
    return wrapper
```

Usage:

```python
@log_call
def add(a, b):
    return a + b

print(add(3, 5))
```

Explanation:

* Logging logic is reusable
* Business logic remains clean
* Behavior is extended transparently

---

## What Is Decorator Chaining?

Decorator chaining means applying **multiple decorators** to a single function.

```python
@decorator_one
@decorator_two
def my_function():
    pass
```

Decorators are applied **bottom-up**, not top-down.

---

## Understanding the Order of Chained Decorators

```python
def decorator_one(func):
    def wrapper():
        print("Decorator One - Before")
        func()
        print("Decorator One - After")
    return wrapper

def decorator_two(func):
    def wrapper():
        print("Decorator Two - Before")
        func()
        print("Decorator Two - After")
    return wrapper

@decorator_one
@decorator_two
def task():
    print("Executing task")
```

Calling `task()` produces:

```
Decorator One - Before
Decorator Two - Before
Executing task
Decorator Two - After
Decorator One - After
```

Explanation:

* `decorator_two` wraps `task` first
* `decorator_one` wraps the result
* Execution enters outer wrapper first and exits last

Internally, this is equivalent to:

```python
task = decorator_one(decorator_two(task))
```

---

## Why Parameterized Decorators Are Needed

Sometimes decorators need configuration:

* Log level
* Retry count
* Permission role
* Timeout duration

A normal decorator cannot accept extra arguments directly.

Parameterized decorators solve this by adding one extra function layer.

---

## Structure of a Parameterized Decorator

A parameterized decorator has **three levels**:

1. Configuration function (receives decorator arguments)
2. Decorator function (receives the function)
3. Wrapper function (wraps execution)

---

## Simple Parameterized Decorator Example

```python
def repeat(times):
    def decorator(func):
        def wrapper():
            for _ in range(times):
                func()
        return wrapper
    return decorator
```

Usage:

```python
@repeat(3)
def greet():
    print("Hello")
```

Explanation:

* `repeat(3)` runs first and returns a decorator
* That decorator wraps the function
* The wrapper controls how many times the function runs

---

## Parameterized Decorator with Arguments and Return Values

```python
def validate_non_empty(label):
    def decorator(func):
        def wrapper(value):
            if not value:
                raise ValueError(f"{label} cannot be empty")
            return func(value)
        return wrapper
    return decorator
```

Usage:

```python
@validate_non_empty("Username")
def process_username(name):
    return name.upper()
```

Explanation:

* Validation logic is injected before function execution
* The original function remains unchanged
* The decorator adds reusable behavior

---

## Chaining Parameterized and Non-Parameterized Decorators

```python
def log(label):
    def decorator(func):
        def wrapper():
            print(f"Log: {label}")
            func()
        return wrapper
    return decorator

def notify(func):
    def wrapper():
        print("Notification sent")
        func()
    return wrapper

@log("START")
@notify
def secure_action():
    print("Performing secure action")
```

Execution follows the same nesting rules:

* Bottom decorator wraps first
* Top decorator wraps last
* Execution enters from the top and exits in reverse order

---

## Script-Based Demonstration

This code must be saved as `decorator_basics_and_chaining.py` and executed from the terminal using:

```
python decorator_basics_and_chaining.py
```

It should not be run inside a Jupyter Notebook.

```python
def audit(action):
    def decorator(func):
        def wrapper():
            print(f"Audit: {action}")
            func()
        return wrapper
    return decorator

def notify(func):
    def wrapper():
        print("Notification sent")
        func()
    return wrapper

@audit("DELETE")
@notify
def delete_record():
    print("Record deleted")

delete_record()
```

---

## Exercise: From Basics to Advanced Decorators

Create decorators step by step.

Part 1:

* Create a simple decorator that prints a message before a function runs

Part 2:

* Extend it to accept arguments using `*args` and `**kwargs`

Part 3:

* Convert it into a parameterized decorator that accepts a label

Part 4:

* Chain it with another decorator that prints a message after execution

Objective:

* Build decorators incrementally
* Understand wrapping, chaining, and parameterization
* Verify execution order through printed output

---

## Summary and Learning Boundary

Key concepts covered:

* Functions are objects and can be wrapped
* Decorators extend behavior without modifying source code
* The `@` syntax simplifies decorator application
* Chained decorators execute in nested order
* Parameterized decorators add configurable behavior

Topics intentionally not covered here:

* Preserving metadata with `functools.wraps`
* Async decorators
* Class-based decorators
* Advanced decorator debugging

From the next topic onward, the material will continue in the regular advanced format without reintroducing decorator basics.


In [122]:
# Why Abstract Base Classes Exist
# As applications grow, code often needs structure and contracts, not just functionality. When multiple classes are expected to behave in a similar way, there must be a clear rule defining:
# What methods must exist
# What behavior is mandatory
# What can vary between implementations

# Abstract Base Classes (ABCs) provide this structure by defining interfaces in Python.
# They allow expressing what a class must do, without dictating how it does it.

In [123]:
# What Is an Abstract Base Class?
# An Abstract Base Class is a class that:
# - Cannot be instantiated directly
# - Defines methods that must be implemented by subclasses
# - Acts as a formal contract

# Python provides ABCs through the built-in abc module.

In [124]:
from abc import ABC, abstractmethod

class Payment(ABC):
    @abstractmethod
    def pay(self, amount):
        pass


In [126]:
# p = Payment()
# TypeError: Can't instantiate abstract class Payment with abstract method pay

In [127]:
class CreditCardPayment(Payment):
    def pay(self, amount):
        print(f"Paid {amount} using Credit Card")

In [128]:
# Crearte an instance of the subclass
payment = CreditCardPayment()
payment.pay(100)

Paid 100 using Credit Card


In [ ]:
# The abc Module
# The abc module provides:
# ABC: a base class for defining abstract classes
# @abstractmethod: a decorator for declaring required methods
# These tools allow Python to enforce interface-like behavior at runtime.
# 
Creating a Simple Abstract Base Class
from abc import ABC, abstractmethod

class Logger(ABC):

    @abstractmethod
    def log(self, message):
        pass

class FileLogger(Logger):

    def log(self, message):
        print(f"File log: {message}")
file_logger = FileLogger()
file_logger.log("This is a log message.")

In [130]:
# Real-World Perspective: Pluggable Architecture
# ABCs are widely used in:
# Logging frameworks
# Database connectors
# Payment gateways
# Authentication providers
# Plugin systems
# Each implementation follows the same interface while behaving differently internally.

In [ ]:
# Exercise: Designing an Interface with ABCs
# Create an abstract base class for a notification system.

# Requirements:

# Abstract method: send(message)

# Two concrete implementations:

# Email notification
# SMS notification
# Each implementation prints a different message format

# Objective:

# Enforce method implementation
# Prevent instantiation of incomplete classes
# Apply interface-based design
# The solution should clearly demonstrate that:

# Only complete implementations can be instantiated
# All implementations follow the same contract